<a href="https://colab.research.google.com/github/MuhammadIrzam447/NewEncodings/blob/main/Train_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Dataset Preparation

In [1]:
%cd /content/MMLearning/data/imdb/rebuttal

/content/MMLearning/data/imdb/rebuttal


In [3]:
import os
image_file_paths = []
genre_labels = []

image_folder_add = "./mulitmodal/train"
labels_file = './train_label.txt'

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces
        image_path = os.path.join(image_folder_add, filename)
        image_file_paths.append(image_path)
        genre_labels.append(labels)

In [4]:
len(image_file_paths)

15552

In [ ]:
# genre_labels

In [ ]:
# unique_labels_set = set()

# for labels in genre_labels:
#     unique_labels_set.update(labels)

# unique_labels = sorted(list(unique_labels_set))

In [ ]:
# unique_labels

In [ ]:
# len(unique_labels)

In [5]:
from collections import defaultdict

label_counts = defaultdict(int)

for labels in genre_labels:
    for label in labels:
        label_counts[label] += 1

In [6]:
# Convert the defaultdict into a list of tuples
label_count_list = [(label, count) for label, count in label_counts.items()]

# Sort the list of tuples based on counts in descending order
sorted_label_count_list = sorted(label_count_list, key=lambda x: x[1], reverse=True)

# Display the sorted list
for label, count in sorted_label_count_list:
    print(f"{label}: {count}")

print("Total Labels: ", len(label_count_list))

Drama: 8424
Comedy: 5108
Romance: 3226
Thriller: 3113
Crime: 2293
Action: 2155
Adventure: 1611
Horror: 1603
Documentary: 1234
Mystery: 1231
Sci-Fi: 1212
Fantasy: 1162
Family: 978
War: 806
Biography: 788
History: 680
Music: 634
Animation: 586
Musical: 503
Western: 423
Sport: 379
Short: 281
Film-Noir: 202
News: 39
Talk-Show: 2
Reality-TV: 1
Total Labels:  26


In [7]:
min_label_count = 40
valid_labels = [label for label, count in label_counts.items() if count >= min_label_count]
valid_labels = sorted(list(valid_labels))

In [8]:
valid_labels, len(valid_labels)

(['Action',
  'Adventure',
  'Animation',
  'Biography',
  'Comedy',
  'Crime',
  'Documentary',
  'Drama',
  'Family',
  'Fantasy',
  'Film-Noir',
  'History',
  'Horror',
  'Music',
  'Musical',
  'Mystery',
  'Romance',
  'Sci-Fi',
  'Short',
  'Sport',
  'Thriller',
  'War',
  'Western'],
 23)

In [ ]:
# filtered_image_paths = []
# filtered_genre_labels = []

# for image_path, labels in zip(image_file_paths, genre_labels):
#     valid_labels_for_sample = [label for label in labels if label in valid_labels]

#     if valid_labels_for_sample:
#         filtered_image_paths.append(image_path)
#         filtered_genre_labels.append(valid_labels_for_sample)

In [ ]:
# len(filtered_image_paths)

In [ ]:
# len(filtered_genre_labels)

In [9]:
multi_hot_labels = []

for labels in genre_labels:
    multi_hot = [1 if label in labels else 0 for label in valid_labels]
    multi_hot_labels.append(multi_hot)

In [10]:
multi_hot_labels[1]

[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

In [11]:
genre_labels[1]

['Action', 'Crime', 'Drama', 'Thriller']

In [12]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class CustomMultiLabelDataset(Dataset):
    def __init__(self, image_file_paths, multi_encoded_labels, transform=None):
        self.image_file_paths = image_file_paths
        self.multi_encoded_labels = multi_encoded_labels
        self.transform = transform

    def __len__(self):
        return len(self.image_file_paths)

    def __getitem__(self, idx):
        image_path = self.image_file_paths[idx]
        image = Image.open(image_path)

        labels = self.multi_encoded_labels[idx]
        labels = torch.tensor(labels, dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, labels

transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to a larger size
    transforms.CenterCrop((224, 224)),  # Center crop to the desired size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [13]:
# Create a custom dataset
train_dataset = CustomMultiLabelDataset(image_file_paths, multi_hot_labels, transform=transform)

In [14]:
len(train_dataset)

15552

In [15]:
train_dataset[10]

(tensor([[[ 1.5982,  1.7180,  1.5810,  ...,  1.6667,  1.5982,  1.6153],
          [ 1.6495,  1.6838,  1.5982,  ...,  1.6153,  1.5639,  1.6838],
          [ 0.6392,  0.6392,  0.6734,  ...,  0.5364,  0.6221,  0.6734],
          ...,
          [ 1.0331,  1.0502,  1.1700,  ...,  1.1700,  1.2557,  1.2728],
          [ 0.1083,  0.1083,  0.5022,  ...,  0.0227,  0.2624,  0.2624],
          [ 0.1083,  0.1083,  0.5022,  ...,  0.0227,  0.2624,  0.2624]],
 
         [[-0.3375, -0.1800, -0.3200,  ...,  0.6078,  0.5028,  0.5378],
          [-0.2325, -0.2150, -0.3025,  ...,  0.5378,  0.5378,  0.7129],
          [-0.1450, -0.1450, -0.0049,  ...,  0.5728,  0.1176,  0.1702],
          ...,
          [ 0.3452,  0.3627,  0.3803,  ...,  0.0301, -0.1099, -0.0924],
          [ 0.2227,  0.2227,  0.2927,  ...,  0.1352, -0.4601, -0.4601],
          [ 0.2227,  0.2227,  0.2927,  ...,  0.1352, -0.4601, -0.4601]],
 
         [[-1.0201, -0.8458, -0.9853,  ..., -1.1770, -1.1770, -1.1073],
          [-0.9678, -0.8807,

# Test Dataset Preparation

In [16]:
import os

test_image_file_paths = []
test_genre_labels = []

image_folder_add = "./mulitmodal/test"
labels_file = './test_label.txt'

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces

        if not (filename.endswith("_1.png") or filename.endswith("_2.png")):
            image_path = os.path.join(image_folder_add, filename)
            test_image_file_paths.append(image_path)
            test_genre_labels.append(labels)


In [17]:
test_image_file_paths[0:10]

['./mulitmodal/test/0078718.png',
 './mulitmodal/test/0089003.png',
 './mulitmodal/test/0098136.png',
 './mulitmodal/test/0057693.png',
 './mulitmodal/test/0385330.png',
 './mulitmodal/test/0096487.png',
 './mulitmodal/test/1220553.png',
 './mulitmodal/test/1341764.png',
 './mulitmodal/test/0882969.png',
 './mulitmodal/test/0119918.png']

In [18]:
test_multi_hot_labels = []

for labels in test_genre_labels:
    test_multi_hot = [1 if label in labels else 0 for label in valid_labels]
    test_multi_hot_labels.append(test_multi_hot)

In [19]:
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [20]:
test_dataset = CustomMultiLabelDataset(test_image_file_paths, test_multi_hot_labels, transform=val_transform)

In [21]:
len(test_dataset)

7799

# Training

In [22]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import torch
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet101(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(valid_labels))
model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [24]:
# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss

optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 30

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

    # if epoch > 10:
    save_dir = "/content/MMLearning/data/Models/Model-08"
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist
    model_name = str(epoch+1) + "_model.pth"
    save_path = os.path.join(save_dir, model_name)  # Specify the complete path to the model file
    torch.save(model.state_dict(), save_path)

    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)

            # predictions.extend((outputs > 0.5).int().cpu().numpy())
            predictions.extend(F.sigmoid(outputs).cpu().numpy())
            true_labels.extend(labels.int().cpu().numpy())

    predictions = np.array(predictions)
    threshold = 0.5
    predictions = (predictions > threshold).astype(int)

    # accuracy = accuracy_score(true_labels, predictions)
    # precision = precision_score(true_labels, predictions, average='macro')
    # recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    # print(f"Accuracy: {accuracy}")
    # print(f"Precision: {precision}")
    # print(f"Recall: {recall}")
    # print(f"F1-Score: {f1}")
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}, F1-Score: {f1:.4f}")
    # print(classification_report(true_labels, predictions))

Epoch 1/30, Loss: 0.2874373007151816, F1-Score: 0.0406
Epoch 2/30, Loss: 0.27961149744290875, F1-Score: 0.0355
Epoch 3/30, Loss: 0.27606771098365507, F1-Score: 0.0502
Epoch 4/30, Loss: 0.27157322216181107, F1-Score: 0.0432
Epoch 5/30, Loss: 0.2676277219206708, F1-Score: 0.0378
Epoch 6/30, Loss: 0.2650839426083329, F1-Score: 0.0694
